In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
taxi_df = pd.read_csv("taxi_data.csv")
bus_df = pd.read_csv("bus_stations.csv")
subway_df = pd.read_csv("subway_stations.csv")

In [3]:
def parseCoords(regex, x):
    arr = regex.match(x)
    return (float(arr.group(2)), float(arr.group(1)))

In [4]:
regex = re.compile("[ ]*POINT \(([-.0-9]+) ([-.0-9]+)\)")
subway_df['coords'] = subway_df.apply(lambda x : parseCoords(regex, x['the_geom']), axis=1)
bus_df['coords'] = list(zip(bus_df['LATITUDE'], bus_df['LONGITUDE']))

In [5]:
subway_df['type'] = 'subway'
bus_df['type'] = 'bus'

In [6]:
bus_df['location'] = bus_df['Street'] + ' AT BETWEEN ' + bus_df['AT_BETWEEN']
bus_df_stripped = bus_df.filter(['location', 'coords', 'type'], axis=1)
subway_df['location'] = subway_df['NAME']
subway_df_stripped = subway_df.filter(['location', 'coords', 'type'], axis=1)
oepnv_locations = pd.concat([subway_df_stripped, bus_df_stripped], ignore_index=True, sort=False)

In [7]:
oepnv_locations

,location,coords,type
0,Astor Pl,"(40.73005400028978, -73.99106999861966)",subway
1,Canal St,"(40.71880300107709, -74.00019299927328)",subway
2,50th St,"(40.76172799961419, -73.98384899986625)",subway
3,Bergen St,"(40.68086213682956, -73.97499915116808)",subway
4,Pennsylvania Ave,"(40.66471445143568, -73.89488591154061)",subway
5,238th St,"(40.88466700064975, -73.90087000018522)",subway
6,Cathedral Pkwy (110th St),"(40.800581558114956, -73.95806670661364)",subway
7,Kingston - Throop Aves,"(40.67991899941601, -73.94085899871263)",subway
8,65th St,"(40.74971952935675, -73.8987883783301)",subway
9,36th St,"(40.75196004401078, -73.92901818461539)",subway


In [8]:
taxi_df_with_location = taxi_df[taxi_df['Pickup_longitude'].notnull()][taxi_df['Pickup_latitude'].notnull()][taxi_df['Dropoff_longitude'].notnull()][taxi_df['Dropoff_latitude'].notnull()]

/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [9]:
taxi_df_with_location['trip_start_coords'] = list(zip(taxi_df_with_location['Pickup_latitude'], taxi_df_with_location['Pickup_longitude']))
taxi_df_with_location['trip_stop_coords'] = list(zip(taxi_df_with_location['Dropoff_latitude'], taxi_df_with_location['Dropoff_longitude']))

In [10]:
taxi_df_stripped = taxi_df_with_location.filter(['Passenger_count', 'Trip_distance', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime', 'Total_amount', 'trip_start_coords', 'trip_stop_coords'], axis=1)
taxi_df_stripped.head(10)

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords
0,1,0.00,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.00,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)"
1,1,2.31,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.30,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)"
2,1,1.79,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.80,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)"
3,1,5.90,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.30,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)"
4,1,0.44,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.30,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)"
5,1,12.49,06/27/2016 08:39:05 PM,06/27/2016 09:05:11 PM,51.41,"(40.69896697998047, -73.80701446533203)","(40.80011367797852, -73.93890380859375)"
6,1,0.67,06/27/2016 08:54:47 PM,06/27/2016 08:58:45 PM,7.88,"(40.71372604370117, -73.82990264892578)","(40.7154426574707, -73.8219223022461)"
7,1,5.16,06/27/2016 08:19:51 PM,06/27/2016 08:42:33 PM,25.56,"(40.67512893676758, -73.95994567871094)","(40.729248046875, -74.00222778320312)"
8,5,3.00,06/18/2016 06:03:15 PM,06/18/2016 06:17:19 PM,13.80,"(40.77011489868164, -73.91835021972656)","(40.74272155761719, -73.88262176513672)"
9,5,0.77,06/18/2016 06:18:20 PM,06/18/2016 06:26:37 PM,7.30,"(40.74272155761719, -73.88262176513672)","(40.73440170288085, -73.87420654296875)"


In [11]:
#calc_min((40.75664138793945, -73.91413116455078), oepnv_locations)
t0 = time.time()
taxi_df_stripped.head(1000)['trip_start_coords'].map(calc_min)
t1 = time.time()
total = t1 - t0
total

NameError: name 'calc_min' is not defined

In [ ]:
30 * 9018 / 60 / 60

In [11]:
def calc_min(x):
    amin = np.argmin([np.linalg.norm(np.array(x)-np.array(y)) for y in oepnv_locations['coords']])
    return oepnv_locations.iloc[amin]['location']

In [12]:
taxi_df_stripped.head(10)
taxi_df_stripped.dtypes

Passenger_count            int64
Trip_distance            float64
lpep_pickup_datetime      object
Lpep_dropoff_datetime     object
Total_amount             float64
trip_start_coords         object
trip_stop_coords          object
dtype: object

In [26]:
taxi_df_stripped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9018030 entries, 0 to 16385531
Data columns (total 7 columns):
Passenger_count          int64
Trip_distance            float64
lpep_pickup_datetime     object
Lpep_dropoff_datetime    object
Total_amount             float64
trip_start_coords        object
trip_stop_coords         object
dtypes: float64(2), int64(1), object(4)
memory usage: 550.4+ MB


In [42]:
from datetime import datetime
weather=pd.read_csv("weather_1.csv",error_bad_lines=False, sep=";")
weather['date']= weather['date'].astype(str)
weather['Date'] = weather['date'].apply(lambda x: datetime.strptime(x, '%d.%m.%y'))
weather = weather.drop("Sum Distance", axis=1)
weather = weather.drop("Time Started", axis=1)
weather = weather.drop("Distance Traveled", axis=1)
weather.head()

,date,Precipitation,Temp_Avg,Date
0,01.01.16,0,"3,3",2016-01-01
1,02.01.16,0,"2,2",2016-01-02
2,03.01.16,0,"4,4",2016-01-03
3,04.01.16,0,"-3,9",2016-01-04
4,05.01.16,0,"-6,7",2016-01-05


In [43]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
date             366 non-null object
Precipitation    366 non-null int64
Temp_Avg         366 non-null object
Date             366 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 11.5+ KB


In [15]:
df_test1= taxi_df_stripped.head(1000)
df_test1['Date'] = pd.to_datetime(df_test1['lpep_pickup_datetime']).dt.date
df_test1['Time'] = pd.to_datetime(df_test1['lpep_pickup_datetime']).dt.time
df_test1['Date']=df_test1['Date'].astype(str)
df_test1['Date']=df_test1["Date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
df_test1.head()


/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a cop

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time
0,1,0.00,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.0,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)",2016-06-27,20:06:54
1,1,2.31,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.3,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)",2016-06-27,20:18:59
2,1,1.79,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.8,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)",2016-06-27,20:41:59
3,1,5.90,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.3,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)",2016-06-27,20:18:37
4,1,0.44,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.3,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)",2016-06-27,20:04:02


In [16]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 9 columns):
Passenger_count          1000 non-null int64
Trip_distance            1000 non-null float64
lpep_pickup_datetime     1000 non-null object
Lpep_dropoff_datetime    1000 non-null object
Total_amount             1000 non-null float64
trip_start_coords        1000 non-null object
trip_stop_coords         1000 non-null object
Date                     1000 non-null datetime64[ns]
Time                     1000 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 78.1+ KB


In [46]:
taxi_df_stripped3 = taxi_df_stripped
taxi_df_stripped3.head()

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords
0,1,0.00,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.0,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)"
1,1,2.31,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.3,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)"
2,1,1.79,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.8,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)"
3,1,5.90,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.3,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)"
4,1,0.44,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.3,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)"


In [47]:
#Split Zeit und Datum in Spalten Zeit und Datum
t0 = time.time()
taxi_df_stripped3['Date'] = pd.to_datetime(taxi_df_stripped3['lpep_pickup_datetime']).dt.date
taxi_df_stripped3['Time'] = pd.to_datetime(taxi_df_stripped3['lpep_pickup_datetime']).dt.time
taxi_df_stripped3['Date']=taxi_df_stripped3['Date'].astype(str)
taxi_df_stripped3['Date']=taxi_df_stripped3["Date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

t1 = time.time()
total = t1 - t0
total

1630.988079071045

0.007539987564086914

In [128]:
# Outer Join von Wetterdaten und unseren Trip Daten
testdf=pd.merge(taxi_df_stripped3, weather, on="Date", how="outer")

#Tempdaten von string mit comma in float mit .
testdf["Temp_Avg"] = [x.replace(',', '.') for x in testdf["Temp_Avg"]]

testdf["Temp_Avg"] = testdf["Temp_Avg"].astype(float)

#Miles in Meter umrechnen
testdf["Trip_distance"]= testdf["Trip_distance"]*1.60934*1000

#Score für Distanz gereist: Wenn Distanz Größer 10000m dann 0,1 Punkte - wenn distanz unter 10000m dann Distanz/1000000
testdf.loc[testdf.Trip_distance > 10000, 'Distance_Score'] = 0.1
testdf.loc[(testdf.Trip_distance <= 10000) & (testdf.Trip_distance >= 1000), 'Distance_Score'] = testdf["Trip_distance"]/100000
testdf.loc[(testdf.Trip_distance < 1000), 'Distance_Score'] = 0.05
testdf.head()




testdf.head(100)

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time,date,Precipitation,Temp_Avg,Distance_Score
0,1.0,0.0000,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.00,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)",2016-06-27,20:06:54,27.06.16,1,23.9,0.050000
1,1.0,3717.5754,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.30,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)",2016-06-27,20:18:59,27.06.16,1,23.9,0.037176
2,1.0,2880.7186,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.80,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)",2016-06-27,20:41:59,27.06.16,1,23.9,0.028807
3,1.0,9495.1060,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.30,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)",2016-06-27,20:18:37,27.06.16,1,23.9,0.094951
4,1.0,708.1096,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.30,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)",2016-06-27,20:04:02,27.06.16,1,23.9,0.050000
5,1.0,20100.6566,06/27/2016 08:39:05 PM,06/27/2016 09:05:11 PM,51.41,"(40.69896697998047, -73.80701446533203)","(40.80011367797852, -73.93890380859375)",2016-06-27,20:39:05,27.06.16,1,23.9,0.100000
6,1.0,1078.2578,06/27/2016 08:54:47 PM,06/27/2016 08:58:45 PM,7.88,"(40.71372604370117, -73.82990264892578)","(40.7154426574707, -73.8219223022461)",2016-06-27,20:54:47,27.06.16,1,23.9,0.010783
7,1.0,8304.1944,06/27/2016 08:19:51 PM,06/27/2016 08:42:33 PM,25.56,"(40.67512893676758, -73.95994567871094)","(40.729248046875, -74.00222778320312)",2016-06-27,20:19:51,27.06.16,1,23.9,0.083042
8,1.0,1367.9390,06/27/2016 08:42:35 PM,06/27/2016 08:46:20 PM,7.56,"(40.76191711425781, -73.92545318603516)","(40.76916122436523, -73.9342041015625)",2016-06-27,20:42:35,27.06.16,1,23.9,0.013679
9,1.0,2253.0760,06/27/2016 08:18:56 PM,06/27/2016 08:26:46 PM,8.80,"(40.72140121459961, -73.84432220458984)","(40.70916366577149, -73.82825469970702)",2016-06-27,20:18:56,27.06.16,1,23.9,0.022531


In [151]:
#time score

# alle fahrten zwischen 6 uhr morgens und 23 uhr abends werden mit 0.1 punkten bepreist


testdf2=testdf.head(1000)
#testdf2['Time2']=testdf2['Time'].astype(str)
#testdf2['Time2']=testdf2['Time'].astype(float)
#testdf2['Time2']=testdf2["Time2"].apply(lambda x: datetime.strptime(x, '%H:%M:%S') if x != 'nan' else None)
#testdf2["Time2"]= testdf2['Time2'].datetime.strptime('%H:%M:%S')
testdf2['Time2']=testdf2['Time'].astype(str)
testdf2['Time2']=testdf2["Time2"].apply(lambda x: datetime.strptime(x, '%H:%M:%S') if x != 'nan' else None)

/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/yannickfischer/anaconda3/envs/learning_env2/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [152]:
testdf2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 16 columns):
Passenger_count          1000 non-null float64
Trip_distance            1000 non-null float64
lpep_pickup_datetime     1000 non-null object
Lpep_dropoff_datetime    1000 non-null object
Total_amount             1000 non-null float64
trip_start_coords        1000 non-null object
trip_stop_coords         1000 non-null object
Date                     1000 non-null datetime64[ns]
Time                     1000 non-null object
date                     1000 non-null object
Precipitation            1000 non-null int64
Temp_Avg                 1000 non-null float64
Distance_Score           1000 non-null float64
Weather_Score            1000 non-null float64
Time_score               0 non-null float64
Time2                    1000 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(7), int64(1), object(6)
memory usage: 132.8+ KB


In [153]:
testdf2.head()

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time,date,Precipitation,Temp_Avg,Distance_Score,Weather_Score,Time_score,Time2
0,1.0,0.0000,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.0,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)",2016-06-27,20:06:54,27.06.16,1,23.9,0.050000,0.0,NaN,1900-01-01 20:06:54
1,1.0,3717.5754,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.3,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)",2016-06-27,20:18:59,27.06.16,1,23.9,0.037176,0.0,NaN,1900-01-01 20:18:59
2,1.0,2880.7186,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.8,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)",2016-06-27,20:41:59,27.06.16,1,23.9,0.028807,0.0,NaN,1900-01-01 20:41:59
3,1.0,9495.1060,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.3,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)",2016-06-27,20:18:37,27.06.16,1,23.9,0.094951,0.0,NaN,1900-01-01 20:18:37
4,1.0,708.1096,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.3,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)",2016-06-27,20:04:02,27.06.16,1,23.9,0.050000,0.0,NaN,1900-01-01 20:04:02


In [155]:
#Wetter Score
testdf.loc[testdf.Precipitation == 1, 'Weather_Score'] = 0
testdf.loc[(testdf.Precipitation == 0) & (testdf.Temp_Avg > 12 ), "Weather_Score"] = 0.1
testdf.head()

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time,date,Precipitation,Temp_Avg,Distance_Score,Weather_Score,Time_score
0,1.0,0.0000,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.0,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)",2016-06-27,20:06:54,27.06.16,1,23.9,0.050000,0.0,NaN
1,1.0,3717.5754,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.3,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)",2016-06-27,20:18:59,27.06.16,1,23.9,0.037176,0.0,NaN
2,1.0,2880.7186,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.8,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)",2016-06-27,20:41:59,27.06.16,1,23.9,0.028807,0.0,NaN
3,1.0,9495.1060,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.3,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)",2016-06-27,20:18:37,27.06.16,1,23.9,0.094951,0.0,NaN
4,1.0,708.1096,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.3,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)",2016-06-27,20:04:02,27.06.16,1,23.9,0.050000,0.0,NaN


In [104]:
#Walking Score
testdf.loc[(testdf.Walking_Distance > 1000) , "Walking_Score"] = 0
testdf.loc[(testdf.Walking_Distance < 1000) , "Walking_Score"] = 0.5 -(testdf["Walking_Distance"]/2000)

AttributeError: 'DataFrame' object has no attribute 'Walking_Distance'

In [138]:
#Time Score
#t0 = time.time()
#testdf2=testdf.head(900000)
#testdf2['Time']=testdf2['Time'].astype(float)
#testdf2['Time']=testdf2["Time"].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
#calc_min((40.75664138793945, -73.91413116455078), oepnv_locations)

time1 = datetime.strptime(a,"%H:%M:%S") # convert string to time
time2 = datetime.strptime(b,"%H:%M:%S") 

#t1 = time.time()
#total = t1 - t0
#total
#testdf.loc[(testdf.Time == "20:06:54") , "Time_score"] = 0
#testdf.head()

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time,date,Precipitation,Temp_Avg,Distance_Score,Weather_Score,Time_score
0,1.0,0.0000,06/27/2016 08:06:54 PM,06/27/2016 08:06:56 PM,20.0,"(40.8540153503418, -73.9371337890625)","(40.85404586791992, -73.93716430664062)",2016-06-27,20:06:54,27.06.16,1,23.9,0.050000,0.0,NaN
1,1.0,3717.5754,06/27/2016 08:18:59 PM,06/27/2016 08:31:30 PM,12.3,"(40.73588180541992, -73.87599945068358)","(40.74331283569336, -73.91846466064453)",2016-06-27,20:18:59,27.06.16,1,23.9,0.037176,0.0,NaN
2,1.0,2880.7186,06/27/2016 08:41:59 PM,06/27/2016 08:51:59 PM,11.8,"(40.74348068237305, -73.92293548583984)","(40.73776626586913, -73.90747833251955)",2016-06-27,20:41:59,27.06.16,1,23.9,0.028807,0.0,NaN
3,1.0,9495.1060,06/27/2016 08:18:37 PM,06/27/2016 08:38:13 PM,21.3,"(40.73053741455078, -73.86274719238281)","(40.75696182250977, -73.9418411254883)",2016-06-27,20:18:37,27.06.16,1,23.9,0.094951,0.0,NaN
4,1.0,708.1096,06/27/2016 08:04:02 PM,06/27/2016 08:06:49 PM,5.3,"(40.74653625488281, -73.89139556884766)","(40.74052047729492, -73.8902359008789)",2016-06-27,20:04:02,27.06.16,1,23.9,0.050000,0.0,NaN


In [127]:

testdf.tail()


,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords,Date,Time,date,Precipitation,Temp_Avg,Distance_Score,Weather_Score
9018209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-27,NaN,27.12.16,0,10.0,NaN,NaN
9018210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-28,NaN,28.12.16,0,2.8,NaN,NaN
9018211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-29,NaN,29.12.16,1,4.2,NaN,0.0
9018212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-30,NaN,30.12.16,1,2.5,NaN,0.0
9018213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-31,NaN,31.12.16,0,3.1,NaN,NaN


In [154]:
testdf.to_csv('Data_wetter_scores.csv')